In [ ]:
from librosa import load 
from librosa import feature
import soundfile
import os, glob, pickle
import numpy as np
import resampy
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def extract_feature(file_name, mfcc=True, chroma=True, mel=True):
    # Assuming X is the audio signal loaded from the file
    X, sample_rate = load(file_name, res_type='kaiser_fast')

    result = np.array([])

    if mfcc:
        mfccs = np.mean(feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))

    if chroma:
        chroma = np.mean(feature.chroma_stft(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, chroma))

    if mel:
        mel = np.mean(feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
        result = np.hstack((result, mel))

    return result


In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
# Assuming emotions and observed_emotions are defined somewhere in your code

# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x, y = [], []

    # Update the directory path to match your dataset
    for file in glob.glob("/home/anshu/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]

        if emotion not in observed_emotions:
            continue

        # Assuming extract_feature is correctly implemented
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)

    # Split the data into training and testing sets
    print("File Names:", [os.path.basename(file) for file in glob.glob("/home/anshu/speech-emotion-recognition-ravdess-data/Actor_*/*.wav")])
    print("Number of samples before split:", len(x))
    return train_test_split(x, y, test_size=test_size, random_state=9)


In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
#Get the shape of the training and testing datasets
print((x_train[0], x_test[0]))

In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train[1]}')

In [ ]:
#Train the model
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(x_train,y_train)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy

accuracy = int(accuracy*100)

print(f"Accuracy:{accuracy}" )



In [ ]:

if accuracy > 55:
    
    print("Observed emotions are : ",observed_emotions)
    
else:
    
    print("Observed emotions are not observed by the classifier ")
    